In [2]:
import os
import pickle
import pandas as pd
import numpy as np
from sqlalchemy import text

In [3]:
# Cargar el MinMaxScaler, el OneHotEncoder, y el modelo guardados
scaler_path = "../00.data/clustering/scaler.pkl"
encoder_path = "../00.data/clustering/encoder.pkl"
ml_clustering_path = "../00.data/clustering/ml_clustering.pkl"

with open(scaler_path, "rb") as f:
    scaler = pickle.load(f)

with open(encoder_path, "rb") as f:
    encoder = pickle.load(f)

with open(ml_clustering_path, "rb") as f:
    rf_model = pickle.load(f)

In [4]:
current_dir = os.path.abspath(os.getcwd())
print(f"Directorio actual: {current_dir}")

# Subir hasta el directorio raíz común (en este caso 'Analisis-de-noticias')
root_dir = os.path.abspath(os.path.join(current_dir, "../.."))
print(f"Directorio raíz común: {root_dir}")

# Construir la ruta al directorio donde están los archivos pkl
directory = os.path.join(root_dir, "src/00.data/preprocesado/")
print(f"Ruta de directorio ajustada: {directory}")

# Lista para almacenar los DataFrames
df_lista = []

# Buscar todos los archivos pkl en el directorio
archivos_pkl = [f for f in os.listdir(directory) if f.startswith("meneame_procesado_") and f.endswith(".pkl")]

# Leer cada archivo .pkl y agregarlo a la lista de DataFrames
for archivo in archivos_pkl:
    file_path = os.path.join(directory, archivo)
    with open(file_path, "rb") as f:
        df_chunk = pickle.load(f)
        df_lista.append(df_chunk)
        print(f"Cargado: {archivo} con {len(df_chunk)} filas")

# Concatenar todos los DataFrames en uno solo
df = pd.concat(df_lista, ignore_index=True)

# Verificar el tamaño del DataFrame final
print(f"DataFrame final con {df.shape[0]} filas y {df.shape[1]} columnas")

Directorio actual: C:\Users\Jordi\OneDrive - Finergal\Documentos\Bootcamp HAB Data Science\PFB\Analisis-de-noticias\src\03_01.ML
Directorio raíz común: C:\Users\Jordi\OneDrive - Finergal\Documentos\Bootcamp HAB Data Science\PFB\Analisis-de-noticias
Ruta de directorio ajustada: C:\Users\Jordi\OneDrive - Finergal\Documentos\Bootcamp HAB Data Science\PFB\Analisis-de-noticias\src/00.data/preprocesado/
Cargado: meneame_procesado_1.pkl con 100000 filas
Cargado: meneame_procesado_2.pkl con 100000 filas
Cargado: meneame_procesado_3.pkl con 87802 filas
DataFrame final con 287802 filas y 19 columnas


In [5]:
df_nuevo = df[df["clicks"].isna()]
df_nuevo.shape

(80665, 19)

In [6]:
df_nuevo.columns

Index(['news_id', 'title', 'content', 'full_story_link', 'meneos', 'clicks',
       'karma', 'positive_votes', 'anonymous_votes', 'negative_votes',
       'category', 'comments', 'published_date', 'user', 'source',
       'source_link', 'scraped_date', 'provincia', 'comunidad'],
      dtype='object')

In [7]:
# **1. Crear una copia sin 'full_story_link' para conservar todas las demás columnas**
df_resultado = df_nuevo.drop(columns=['full_story_link'])

In [8]:
features_numericas = ['meneos', 'karma', 'positive_votes', 'anonymous_votes', 'negative_votes', 'comments']
feature_categorica = 'category'

# Verificar que las columnas existen en el DataFrame
if not all(col in df_nuevo.columns for col in features_numericas + [feature_categorica]):
    raise ValueError("Faltan columnas necesarias en el DataFrame.")

# Escalar las variables numéricas
df_scaled = scaler.transform(df_nuevo[features_numericas])

# Codificar la variable categórica
encoded_categorias = encoder.transform(df_nuevo[[feature_categorica]])
encoded_categorias_df = pd.DataFrame(encoded_categorias, columns=encoder.get_feature_names_out([feature_categorica]))

# Unir los datos transformados
df_final = pd.concat([pd.DataFrame(df_scaled, columns=features_numericas), encoded_categorias_df], axis=1)

# Asegurar que las columnas coinciden con las del modelo
df_final = df_final.reindex(columns=rf_model.feature_names_in_, fill_value=0)

# **3. Predecir los clusters**
df_resultado["cluster"] = rf_model.predict(df_final)

# **4. Guardar el resultado**
df_resultado.to_csv("../00.data/clustering/scraped_news_with_clusters.csv", index=False)

print("Predicción de clusters completada y guardada.")

Predicción de clusters completada y guardada.


In [9]:
df_resultado.head(3)

,news_id,title,content,meneos,clicks,karma,positive_votes,anonymous_votes,negative_votes,category,comments,published_date,user,source,source_link,scraped_date,provincia,comunidad,cluster
150874,1037095,"Topless: mirar o no mirar, ésa es la cuestión",Acabo de retornar de un roadtrip veraniego. Bi...,512,NaN,635,217,295,3,Entretenimiento y Cultura,95,2010-08-31 19:00:02,Kfn,elreferente.es,http://www.elreferente.es/polmiraflors/topless...,2025-02-27 11:25:53,NaN,NaN,0
150875,1034957,Elogio del porno,Saber follar es importante pero hay algo que i...,430,NaN,632,180,250,4,Entretenimiento y Cultura,48,2010-08-28 17:25:03,--98342--,lne.es,http://www.lne.es/gijon/2010/08/27/elogio-porn...,2025-02-27 11:26:04,NaN,NaN,0
150876,1026190,El 56% de hombres rechaza el topless de sus no...,Un 56% de hombres no está de acuerdo con su pa...,793,NaN,645,269,524,6,Política y Sociedad,100,2010-08-16 00:45:02,cd_autoreverse,noticias24.com,http://www.noticias24.com/gente/noticia/10295/...,2025-02-27 11:26:56,NaN,NaN,0


In [10]:
df_resultado['cluster'].value_counts()

cluster
0    73970
2     5947
1      748
Name: count, dtype: int64

In [11]:
# Guardar el df
clustering_path = "../00.data/clustering/df_clustering_null_clicks.pkl"

with open(clustering_path, "wb") as f:
    pickle.dump(df_resultado, f, protocol=pickle.HIGHEST_PROTOCOL)